In [1]:
import numpy as np
import pickle
from learnable_crf import LearnableCrf
from lib import *
from scipy.special import expit as sigmoid

In [2]:
with open('cache/df_val_test.pickle', mode='rb') as h:
    df_val, df_test = pickle.load(h)
Y_val = df_val['label']
Y_test = df_test['label']
with open('cache/hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)
id_fh = map(lambda x: list(max(x, key=len)), hex_data['id_hierarchical_labels'])  # convert to list for addressing
state_space = hex_data['state_space']
# state_space = filter(lambda x: x[:20].any(), state_space)

In [3]:
r = 50
with open('cache/df_train.{}.pickle'.format(r), mode='rb') as h:
    df = pickle.load(h)
leaves = np.nonzero([x[0] in x[1] for x in zip(df['label'], df['pseudo_label'])])[0]
Y_train = df['label'][leaves]

In [4]:
model_Phi_train = np.load('results/caffe_independent/iter_Phi_train.{}.npy'.format(r))[:, leaves]
model_Phi_val = np.load('results/caffe_independent/iter_Phi_val.{}.npy'.format(r))
model_Phi_test = np.load('results/caffe_independent/iter_Phi_test.{}.npy'.format(r))
M = len(model_Phi_train)

In [5]:
def confusion_matrix(Y_predict, Y_truth):
    if Y_predict.dtype == bool:
        cm = np.zeros((27, 27), dtype=int)
        count = np.zeros(27, dtype=int)
        for i, y in enumerate(Y_predict):
            count[id_fh[Y_truth[i]]] += 1
            cm[id_fh[Y_truth[i]], :] += y
    else:
        cm = np.zeros((20, 27), dtype=int)
        count = np.zeros(20, dtype=int)
        for i, y in enumerate(Y_predict):
            count[Y_truth[i]] += 1
            cm[Y_truth[i], y.argmax()] += 1
    return cm.astype(float) / count[:, None]

In [6]:
def raw_crf(Phi):
    def step(phi):
        scores = map(lambda s: phi[s].sum(), state_space)
#         return np.vstack(tuple(state_space[np.argsort(scores)[i]] for i in range(-5, 0)))
        return state_space[np.argmax(scores)]
    return np.array(map(step, Phi), dtype=bool)

In [7]:
[get_accuracy(model_Phi_val[i], Y_val, lim_states=False) for i in range(0, M)]
# [get_accuracy(raw_crf(model_Phi_val[i]), Y_val) for i in range(0, M)]

[0.004156769596199525,
 0.004156769596199525,
 0.0059382422802850355,
 0.006532066508313539,
 0.006532066508313539,
 0.006532066508313539,
 0.006532066508313539,
 0.006532066508313539,
 0.006532066508313539,
 0.006532066508313539]

In [8]:
[get_accuracy(model_Phi_test[i], Y_test, lim_states=False) for i in range(0, M)]
# [get_accuracy(raw_crf(model_Phi_test[i]), Y_test) for i in range(0, M)]

[0.00831353919239905,
 0.0029691211401425177,
 0.004156769596199525,
 0.004750593824228029,
 0.004750593824228029,
 0.004750593824228029,
 0.004750593824228029,
 0.004750593824228029,
 0.004750593824228029,
 0.004750593824228029]

In [12]:
np.diagonal(confusion_matrix(model_Phi_test[-1], Y_test))
# np.diagonal(confusion_matrix(raw_crf(model_Phi_test[-1]), Y_test))

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.00689655,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.00943396,  0.02985075])

In [10]:
model_Phi_train = sigmoid(model_Phi_train)
model_Phi_val = sigmoid(model_Phi_val)
model_Phi_test = sigmoid(model_Phi_test)
lcrf = [LearnableCrf(model_Phi_train[i], Y_train) for i in range(0, M)]
[get_accuracy(lcrf[i].predict(model_Phi_val[i]), Y_val) for i in range(0, M)]

[0.39667458432304037,
 0.43646080760095013,
 0.4667458432304038,
 0.4637767220902613,
 0.4649643705463183,
 0.4655581947743468,
 0.4655581947743468,
 0.4655581947743468,
 0.4655581947743468,
 0.4655581947743468]

In [11]:
[get_accuracy(lcrf[i].predict(model_Phi_test[i]), Y_test) for i in range(0, M)]

[0.3812351543942993,
 0.43764845605700714,
 0.46140142517814725,
 0.46140142517814725,
 0.46199524940617576,
 0.4637767220902613,
 0.4643705463182898,
 0.4643705463182898,
 0.4643705463182898,
 0.4643705463182898]

In [13]:
np.diagonal(confusion_matrix(lcrf[-1].predict(model_Phi_test[-1]), Y_test))

array([ 0.46428571,  0.11111111,  0.13793103,  0.3       ,  0.10714286,
        0.475     ,  0.56666667,  0.82352941,  0.453125  ,  0.85185185,
        0.48214286,  0.45517241,  0.2739726 ,  0.05769231,  0.69565217,
        0.5       ,  0.50568182,  0.32413793,  0.55660377,  0.53731343,
        0.76264591,  0.67295597,  0.95496894,  0.93896714,  0.87356322,
        0.69677419,  0.85480094])

In [14]:
lcrf[-1].opt_theta

array([ 2.18251083,  1.31984708,  2.16087463,  0.618895  ,  0.92283248,
        0.70104614,  2.1050899 ,  0.        ,  1.18143035,  0.        ,
        2.19203059,  2.13827253,  1.72629435,  1.5891431 ,  0.        ,
        0.        ,  1.99344555,  2.07551518,  2.73015487,  1.39382511,
        0.79709192,  1.68151799,  0.30984177,  2.90600773,  2.47913771,
        0.        ,  3.99403071,  0.98459505,  1.01912682,  0.95826632,
        0.50581091,  0.96929608,  0.98769787,  0.96239557,  0.89441708,
        0.95424326,  2.20208736,  0.88772576,  1.12623125,  0.91805882,
        0.97884967,  0.93246386,  0.91195274,  0.51732685,  1.80395428,
        0.98574547,  0.96432693,  0.98211448,  0.84280092,  0.88765158,
        2.75150097])